# Sharadar Data Explorer

This notebook helps you explore the Sharadar bundle data:
1. **Database Summary** - Overview of available data
2. **Price Data** - Explore OHLCV pricing for specific symbols
3. **Fundamentals Data** - Explore SF1 fundamentals for specific symbols
4. **Field Availability** - See which fields have data
5. **Symbol Lookup** - Find SIDs and date ranges for symbols

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os

# Register bundle
from zipline.data.bundles import register
from zipline.data.bundles.sharadar_bundle import sharadar_bundle

register('sharadar', sharadar_bundle(tickers=None, incremental=True, include_funds=True))

# Load bundle
from zipline.data.bundles import load as load_bundle
bundle_data = load_bundle('sharadar')

print("✓ Sharadar bundle loaded successfully")
print(f"Bundle path: {bundle_data.equity_daily_bar_reader._rootdir}")

## 1. Database Summary

In [ ]:
# Get all assets
all_assets = bundle_data.asset_finder.retrieve_all(bundle_data.asset_finder.sids)

print("="*80)
print("SHARADAR BUNDLE SUMMARY")
print("="*80)
print(f"\nTotal Assets: {len(all_assets):,}")

# Date range
start_dates = [asset.start_date for asset in all_assets if asset.start_date]
end_dates = [asset.end_date for asset in all_assets if asset.end_date]

if start_dates and end_dates:
    print(f"Date Range: {min(start_dates).date()} to {max(end_dates).date()}")
    print(f"  Earliest data: {min(start_dates).date()}")
    print(f"  Latest data: {max(end_dates).date()}")

# Asset types
asset_types = {}
for asset in all_assets:
    asset_type = getattr(asset, 'asset_type', 'equity')
    asset_types[asset_type] = asset_types.get(asset_type, 0) + 1

print(f"\nAsset Types:")
for asset_type, count in sorted(asset_types.items()):
    print(f"  {asset_type}: {count:,}")

# Sample assets
print(f"\nSample Assets:")
for asset in list(all_assets)[:10]:
    print(f"  {asset.symbol:8s} SID: {asset.sid:6d}  {asset.start_date.date()} to {asset.end_date.date()}")

## 2. Symbol Lookup and Info

In [ ]:
def get_asset_info(symbol, as_of_date=None):
    """Get detailed information about a symbol."""
    try:
        asset = bundle_data.asset_finder.lookup_symbol(symbol, as_of_date=as_of_date)
        
        print("="*80)
        print(f"ASSET INFO: {symbol}")
        print("="*80)
        print(f"Symbol: {asset.symbol}")
        print(f"SID: {asset.sid}")
        print(f"Asset Name: {asset.asset_name}")
        print(f"Exchange: {asset.exchange}")
        print(f"Start Date: {asset.start_date.date()}")
        print(f"End Date: {asset.end_date.date()}")
        print(f"Asset Type: {getattr(asset, 'asset_type', 'equity')}")
        
        return asset
    except Exception as e:
        print(f"❌ Error looking up {symbol}: {e}")
        return None

# Example: Look up AAPL
aapl = get_asset_info('AAPL')

In [ ]:
# Look up multiple symbols
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA']

print("="*80)
print("SYMBOL LOOKUP TABLE")
print("="*80)
print(f"{'Symbol':<10} {'SID':<8} {'Start Date':<12} {'End Date':<12} {'Days':<8}")
print("-"*80)

asset_map = {}
for symbol in symbols:
    try:
        asset = bundle_data.asset_finder.lookup_symbol(symbol, as_of_date=None)
        asset_map[symbol] = asset
        days = (asset.end_date - asset.start_date).days
        print(f"{asset.symbol:<10} {asset.sid:<8} {asset.start_date.date()} {asset.end_date.date()} {days:<8,}")
    except Exception as e:
        print(f"{symbol:<10} Not found: {e}")

## 3. Price Data Explorer

In [ ]:
def get_price_data(symbol, start_date=None, end_date=None):
    """Get OHLCV price data for a symbol."""
    try:
        # Look up asset
        asset = bundle_data.asset_finder.lookup_symbol(symbol, as_of_date=None)
        
        # Get sessions (trading days) - these are tz-naive
        sessions = bundle_data.equity_daily_bar_reader.sessions
        
        # Default dates
        if start_date is None:
            start_date = asset.start_date
        else:
            start_date = pd.Timestamp(start_date)
        
        if end_date is None:
            end_date = asset.end_date
        else:
            end_date = pd.Timestamp(end_date)
        
        # Remove timezone for comparison with tz-naive sessions
        if start_date.tz is not None:
            start_date = start_date.tz_localize(None)
        if end_date.tz is not None:
            end_date = end_date.tz_localize(None)
        
        # Filter sessions by date range
        date_range = sessions[(sessions >= start_date) & (sessions <= end_date)]
        
        if len(date_range) == 0:
            print(f"⚠️  No data available for {symbol} in date range")
            return None
        
        # Load data
        data = bundle_data.equity_daily_bar_reader.load_raw_arrays(
            columns=['open', 'high', 'low', 'close', 'volume'],
            start_date=date_range[0],
            end_date=date_range[-1],
            sids=[asset.sid],
        )
        
        # Create DataFrame
        df = pd.DataFrame({
            'open': data[0][:, 0],
            'high': data[1][:, 0],
            'low': data[2][:, 0],
            'close': data[3][:, 0],
            'volume': data[4][:, 0],
        }, index=date_range)
        
        # Remove rows with NaN (no data)
        df = df.dropna()
        
        print(f"✓ Loaded {len(df):,} days of price data for {symbol}")
        print(f"  Date range: {df.index[0].date()} to {df.index[-1].date()}")
        
        return df
    
    except Exception as e:
        print(f"❌ Error loading price data for {symbol}: {e}")
        import traceback
        traceback.print_exc()
        return None

# Example: Get AAPL price data for 2023
aapl_prices = get_price_data('AAPL', '2023-01-01', '2023-12-31')

if aapl_prices is not None:
    print("\nSample data:")
    print(aapl_prices.head(10))

In [ ]:
# Plot price data
if aapl_prices is not None and len(aapl_prices) > 0:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Price
    ax1.plot(aapl_prices.index, aapl_prices['close'], label='Close', linewidth=2)
    ax1.fill_between(aapl_prices.index, aapl_prices['low'], aapl_prices['high'], alpha=0.3, label='High-Low Range')
    ax1.set_ylabel('Price ($)', fontsize=12)
    ax1.set_title('AAPL Price History (2023)', fontsize=14, fontweight='bold')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Volume
    ax2.bar(aapl_prices.index, aapl_prices['volume'], alpha=0.7, color='steelblue')
    ax2.set_ylabel('Volume', fontsize=12)
    ax2.set_xlabel('Date', fontsize=12)
    ax2.set_title('Trading Volume', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e6:.0f}M'))
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print("\n" + "="*80)
    print("PRICE STATISTICS (2023)")
    print("="*80)
    print(aapl_prices.describe())

## 4. Fundamentals Data Explorer

Explore the SF1 fundamentals data (if available).

In [ ]:
# Check if fundamentals file exists
bundle_path = Path(bundle_data.equity_daily_bar_reader._rootdir)
sf1_path = bundle_path / 'fundamentals' / 'sf1.h5'

print(f"Checking for fundamentals data...")
print(f"Path: {sf1_path}")
print(f"Exists: {sf1_path.exists()}")

if sf1_path.exists():
    print(f"File size: {sf1_path.stat().st_size / 1024 / 1024:.2f} MB")
    FUNDAMENTALS_AVAILABLE = True
else:
    print("⚠️  Fundamentals not available. Run ingest with include_fundamentals=True")
    FUNDAMENTALS_AVAILABLE = False

In [ ]:
if FUNDAMENTALS_AVAILABLE:
    # Load fundamentals data
    with pd.HDFStore(sf1_path, 'r') as store:
        print("\nHDF5 Keys:")
        for key in store.keys():
            print(f"  {key}")
        
        # Load the main table
        sf1 = store['sf1']
        
        print(f"\n✓ Loaded SF1 fundamentals")
        print(f"  Shape: {sf1.shape}")
        print(f"  Columns: {len(sf1.columns)}")
        print(f"\nFirst few columns: {list(sf1.columns)[:20]}")

In [ ]:
if FUNDAMENTALS_AVAILABLE:
    def get_fundamentals(symbol, field='revenue'):
        """Get fundamentals data for a specific symbol and field."""
        # Look up SID
        asset = bundle_data.asset_finder.lookup_symbol(symbol, as_of_date=None)
        
        # Filter for this SID
        data = sf1[sf1['sid'] == asset.sid].copy()
        
        if len(data) == 0:
            print(f"⚠️  No fundamentals data for {symbol} (SID {asset.sid})")
            return None
        
        # Convert dates
        data['calendardate'] = pd.to_datetime(data['calendardate'])
        data = data.sort_values('calendardate')
        
        print(f"✓ Found {len(data):,} fundamental records for {symbol}")
        print(f"  Date range: {data['calendardate'].min().date()} to {data['calendardate'].max().date()}")
        print(f"  Dimensions: {data['dimension'].unique()}")
        
        return data
    
    # Example: Get AAPL fundamentals
    aapl_fund = get_fundamentals('AAPL')
    
    if aapl_fund is not None:
        print("\nSample records:")
        print(aapl_fund[['calendardate', 'dimension', 'revenue', 'netinc', 'fcf', 'equity', 'debt']].tail(10))

In [ ]:
if FUNDAMENTALS_AVAILABLE and aapl_fund is not None:
    # Filter for Most Recent Quarter (MRQ)
    aapl_quarterly = aapl_fund[aapl_fund['dimension'] == 'MRQ'].copy()
    
    # Plot key metrics over time
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Revenue
    axes[0, 0].plot(aapl_quarterly['calendardate'], aapl_quarterly['revenue'] / 1e9, marker='o')
    axes[0, 0].set_title('Quarterly Revenue', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylabel('Revenue ($B)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Net Income
    axes[0, 1].plot(aapl_quarterly['calendardate'], aapl_quarterly['netinc'] / 1e9, marker='o', color='green')
    axes[0, 1].set_title('Quarterly Net Income', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Net Income ($B)')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Free Cash Flow
    axes[1, 0].plot(aapl_quarterly['calendardate'], aapl_quarterly['fcf'] / 1e9, marker='o', color='orange')
    axes[1, 0].set_title('Quarterly Free Cash Flow', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('FCF ($B)')
    axes[1, 0].set_xlabel('Date')
    axes[1, 0].grid(True, alpha=0.3)
    
    # PE Ratio
    axes[1, 1].plot(aapl_quarterly['calendardate'], aapl_quarterly['pe'], marker='o', color='red')
    axes[1, 1].set_title('P/E Ratio', fontsize=12, fontweight='bold')
    axes[1, 1].set_ylabel('P/E Ratio')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle('AAPL Quarterly Fundamentals Over Time', fontsize=14, fontweight='bold', y=1.00)
    plt.tight_layout()
    plt.show()

## 5. Field Availability Analysis

Check which fundamental fields have data.

In [ ]:
if FUNDAMENTALS_AVAILABLE:
    print("Analyzing field availability...\n")
    print("="*80)
    print("FIELD AVAILABILITY")
    print("="*80)
    
    availability = []
    
    for col in sf1.columns:
        if col in ['ticker', 'dimension', 'calendardate', 'datekey', 'reportperiod', 'fiscalperiod', 'lastupdated', 'sid', 'sharefactor', 'permaticker']:
            continue
        
        non_null = sf1[col].notna().sum()
        pct = non_null / len(sf1) * 100
        availability.append((col, non_null, pct))
    
    # Sort by availability
    availability.sort(key=lambda x: x[2], reverse=True)
    
    print(f"\n{'Field':<30} {'Non-null':<15} {'Availability':>12}")
    print("-"*80)
    
    # Show top 40 fields
    for col, non_null, pct in availability[:40]:
        print(f"{col:<30} {non_null:<15,} {pct:>11.1f}%")
    
    print(f"\n...and {len(availability) - 40} more fields")

In [ ]:
if FUNDAMENTALS_AVAILABLE:
    # Show fields with 0% availability (empty)
    empty_fields = [col for col, count, pct in availability if pct == 0]
    
    if empty_fields:
        print("\n" + "="*80)
        print("EMPTY FIELDS (0% availability)")
        print("="*80)
        for field in empty_fields:
            print(f"  - {field}")
    else:
        print("\n✓ No empty fields found!")

## 6. Compare Multiple Symbols

Compare fundamentals across multiple companies.

In [ ]:
if FUNDAMENTALS_AVAILABLE:
    def compare_fundamentals(symbols, field='revenue', dimension='MRQ', recent_quarters=8):
        """Compare a fundamental metric across multiple symbols."""
        
        fig, ax = plt.subplots(figsize=(14, 6))
        
        for symbol in symbols:
            try:
                asset = bundle_data.asset_finder.lookup_symbol(symbol, as_of_date=None)
                data = sf1[(sf1['sid'] == asset.sid) & (sf1['dimension'] == dimension)].copy()
                
                if len(data) == 0:
                    print(f"⚠️  No data for {symbol}")
                    continue
                
                data['calendardate'] = pd.to_datetime(data['calendardate'])
                data = data.sort_values('calendardate').tail(recent_quarters)
                
                # Plot
                ax.plot(data['calendardate'], data[field] / 1e9, marker='o', label=symbol, linewidth=2)
                
            except Exception as e:
                print(f"❌ Error for {symbol}: {e}")
        
        ax.set_xlabel('Date', fontsize=12)
        ax.set_ylabel(f'{field.title()} ($B)', fontsize=12)
        ax.set_title(f'{field.title()} Comparison - Last {recent_quarters} Quarters', fontsize=14, fontweight='bold')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    # Compare revenue for tech giants
    compare_fundamentals(['AAPL', 'MSFT', 'GOOGL', 'AMZN'], field='revenue', recent_quarters=12)

## 7. Custom Queries

Run your own custom queries on the data.

In [ ]:
# Example: Find stocks with highest ROE (if field is available)
if FUNDAMENTALS_AVAILABLE:
    # Get most recent quarter for each stock
    latest = sf1[sf1['dimension'] == 'MRQ'].copy()
    latest['calendardate'] = pd.to_datetime(latest['calendardate'])
    
    # Get the latest date for each ticker
    latest_by_ticker = latest.sort_values('calendardate').groupby('ticker').tail(1)
    
    # Filter for stocks with positive FCF and reasonable PE
    quality = latest_by_ticker[
        (latest_by_ticker['fcf'] > 0) &
        (latest_by_ticker['pe'] > 0) &
        (latest_by_ticker['pe'] < 50)
    ].copy()
    
    if len(quality) > 0:
        # Sort by FCF
        top_fcf = quality.nlargest(20, 'fcf')
        
        print("\n" + "="*80)
        print("TOP 20 STOCKS BY FREE CASH FLOW (Most Recent Quarter)")
        print("="*80)
        print(f"\n{'Ticker':<10} {'FCF ($B)':<12} {'Revenue ($B)':<12} {'P/E':<8} {'Date':<12}")
        print("-"*80)
        
        for _, row in top_fcf.iterrows():
            print(f"{row['ticker']:<10} {row['fcf']/1e9:<12.2f} {row['revenue']/1e9:<12.2f} {row['pe']:<8.2f} {row['calendardate'].date()}")

## 8. Export Data

Export data to CSV for external analysis.

In [ ]:
# Example: Export AAPL price data
if aapl_prices is not None:
    output_file = 'aapl_prices_2023.csv'
    aapl_prices.to_csv(output_file)
    print(f"✓ Exported AAPL price data to {output_file}")

# Example: Export AAPL fundamentals
if FUNDAMENTALS_AVAILABLE and aapl_fund is not None:
    output_file = 'aapl_fundamentals.csv'
    aapl_fund.to_csv(output_file, index=False)
    print(f"✓ Exported AAPL fundamentals to {output_file}")

## Summary

This notebook provides tools to:
- ✓ Browse available symbols and their SIDs
- ✓ Explore price history (OHLCV)
- ✓ Analyze fundamentals data (if available)
- ✓ Check field availability
- ✓ Compare metrics across companies
- ✓ Export data for further analysis

**Next Steps:**
1. Modify the `symbols` list to explore different companies
2. Change `field` parameter to explore different metrics
3. Adjust date ranges to focus on specific time periods
4. Create custom queries to find interesting stocks